In [7]:
%pip install torch numpy matplotlib tqdm
%pip install typed-argument-parser
%pip install einops gitpython
%pip install google-api-python-client pandas ffmpeg ffprobe pillow wandb
%pip install scikit-video scikit-image
%pip install stable-baselines3 gymnasium minigrid
%pip install triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 4.8 MB/s eta 0:00:00


In [2]:
# install the repo
!git clone --single-branch -b maze2d https://github.com/AJB-ajb/diffuser.git


Cloning into 'diffuser'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 236 (delta 104), reused 120 (delta 93), pack-reused 63 (from 1)
Receiving objects: 100% (236/236), 595.72 KiB | 27.08 MiB/s, done.
Resolving deltas: 100% (110/110), done.


In [3]:
!cd /content/diffuser && git pull
!cd /content/diffuser && pip install -e ../diffuser

import sys
sys.path.append('/content/diffuser/minigrid_scripts')

Already up to date.
Obtaining file:///content/diffuser
  Preparing metadata (setup.py) ... done
  Running setup.py develop for diffuser


In [4]:
# define the config
from pathlib import Path
import mgcfg

cfg = mgcfg.base_cfg
cfg['name'] = "empty_env"
cfg['name'] = "empty_env"
cfg['env_id'] = "MiniGrid-Empty-16x16-v0"
cfg['env_module'] = "empty_env"
cfg['feature_coder'] = "EmptyEnvDiscFC"
cfg['policy']['id'] = "cnn_2e5"
cfg['policy']['n_timesteps'] = 2e5
cfg['collection']['id'] = "policy_random_1000"
cfg['collection']['n_episodes'] = 1000

# for testing
cfg.policy['n_timesteps'] = 1e5
cfg.policy['id'] = 'cnn_1e3'
cfg.policy['name'] = 'CnnPolicy'

# if on google colab, mount drive and set the path
on_drive = True
if on_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    cfg['storage_dir'] = Path("/content/drive/My Drive/MGDiffuser")
    cfg.storage_dir.mkdir(exist_ok = True)

exp = mgcfg.Experiment(cfg)
cfg.save_to_json()

# train the policy
cfg_path = str(cfg.cfg_path)

Mounted at /content/drive


In [20]:
!echo python train_policy.py --cfg "$cfg_path"

python train_policy.py --cfg /content/drive/My Drive/MGDiffuser/cfgs/empty_env.json


In [18]:
!cat "/content/drive/My Drive/MGDiffuser/cfgs/empty_env.json"

{"is_toplevel": true, "name": "empty_env", "env_id": "", "env_module": "", "feature_coder": "", "policy": {"id": "cnn_1e3", "name": "CnnPolicy", "features_extractor_class": "MinigridFeaturesExtractor", "features_dim": 128, "policy_kwargs": {"features_extractor_kwargs": {"features_dim": 128}}, "n_timesteps": 100000.0}, "collection": {"id": "policy_random_1000", "n_episodes": 1000}, "horizon": 128, "dim_mults": [1, 4, 8], "diffusion": {"n_diffusion_steps": 64, "action_weight": 1, "loss_discount": 1, "predict_epsilon": false, "clip_denoised": true, "use_padding": false}, "trainer": {"ema_decay": 0.995, "train_batch_size": 32, "train_lr": 0.0002, "gradient_accumulate_every": 2, "log_freq": 100, "sample_freq": 1000, "save_freq": 1000, "label_freq": 100000, "save_parallel": false, "n_reference": 50, "n_samples": 10, "bucket": null, "n_train_steps": 2000000, "n_steps_per_epoch": 10000}}

In [21]:
# train the policy # takes about 5m for 2e5 timesteps
!cd diffuser/minigrid_scripts/ && python train_policy.py --cfg "$cfg_path"

2024-12-11 17:33:43.472440: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 17:33:43.493182: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 17:33:43.499524: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 17:33:44.543150: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists

In [22]:
# collect trajectories # about 3 mins for collection and 3mins for evaluation for 1000 trajectories
!cd diffuser/minigrid_scripts/ && python collect_trajectories.py --cfg "$cfg_path"

2024-12-11 17:40:59.247501: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 17:40:59.268238: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 17:40:59.274731: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 17:41:00.433694: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
100% 1000/1000 [02:47<00:00,  5.95it/s]
Average state consistency (should be 1.):  1.0
Average action con

In [8]:
# train the diffuser
# standard training steps 2e6; takes about 25 secs per 100 iterations pure training; i.e. 25 * 10000 secs
!cd diffuser/minigrid_scripts/ && python train_diffuser.py --cfg "$cfg_path"

Streaming output truncated to the last 5000 lines.
                                                                                                    
                                                                                                    
24 / 64 [######################                                      ]  37% | 103.2 Hz
t : 40
                                                                                                    
                                                                                                    
25 / 64 [#######################                                     ]  39% | 103.3 Hz
t : 39
                                                                                                    
                                                                                                    
26 / 64 [########################                                    ]  40% | 103.3 Hz
t : 38
                                                             

In [6]:
# get torch version
import torch
print(torch.__version__)

2.5.1+cu121


In [ ]:
# evaluate the diffuser
!cd diffuser/minigrid_scripts/ && python evaluate_diffuser.py --cfg "$cfg_path"